In [ ]:
!pip install tqdm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
# import pytorch
import numpy as np
import time
import shutil

start_time = time.time()

import pandas as pd
import csv
import torchvision
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

from torchvision import datasets, models, transforms
import torch.nn as nn

LABELS_FILE_PATH = "noTreeCropsEmbeddedSides.csv"
IMAGE_FOLDER_PATH = "noTreeCropsEmbeddedSides/"


def loadCSV(LABELS_FILE_PATH):
    filenames = []
    labels = []
    with open(LABELS_FILE_PATH) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:
                filename = row[0]
                idx = filename.find('-')
                # print(filename[idx+1:])
                filenames.append(filename[idx+1:])
                labels.append(row[1])
                line_count += 1
        print(f'Loaded CSV with {line_count} labels.')
    return filenames, labels

def encodeLabels(labels):
    classes = {'Field': 0, 'Not-Field': 1}
    newLabels = []
    for i, l in enumerate(labels):
        if l not in '':
            newLabels.append(classes[l])
        else:
            newLabels.append(1)
            print(i)
    return newLabels

def uploadImages(folderPath, filenames, labels, batchSize):
    data = []

    labels = encodeLabels(labels)

    for i, file in enumerate(filenames):
        if i != 95:
            im = torchvision.io.read_image(folderPath + file)
            # data.append([torchvision.io.read_image(folderPath + file), labels[i]])

            data.append([im.to(torch.float), labels[i]])

    # train_set = datasets.ImageFolder(folderPath)
    splitIdx = round(len(data)*0.7)
    train = DataLoader(data[:splitIdx], batch_size = batchSize)
    test = DataLoader(data[splitIdx:], batch_size = batchSize)

    return train, test

def train_imshow():
    classes = {'Field': 0, 'Not-Field': 1} # Defining the classes we have
    dataiter = iter(trainloader)
    images, labels = dataiter.next()
    print(labels)
    fig, axes = plt.subplots(figsize=(10, 4), ncols=5)
    for i in range(5):
        ax = axes[i]
        ax.imshow(images[i].permute(1, 2, 0))
        ax.title.set_text(' '.join('%5s' % classes[labels[i]]))
    plt.show()

BATCH_SIZE = 32

filenames, labels = loadCSV(LABELS_FILE_PATH)
trainloader, testloader = uploadImages(IMAGE_FOLDER_PATH, filenames, labels, BATCH_SIZE)


Column names are image, choice, created_at, annotation_id, id, lead_time, updated_at, annotator
Loaded CSV with 615 labels.
95


In [42]:
def classBalance(labels):
  encodedLabels = encodeLabels(labels)
  field = 0
  nfield = 0
  for l in encodedLabels:
    if l == 0:
      field +=1
    elif l == 1:
      nfield +=1
  print('Field Images : {}, Non-field Images : {}'.format(field,nfield))
classBalance(labels)

95
Field Images : 240, Non-field Images : 374


In [45]:

# print('Labels', np.array(labels).shape)


# print(np.array(test))

def make_train_step(model, optimizer, loss_fn):
  def train_step(x,y):
    #make prediction
    yhat = model(x)
    #enter train mode
    model.train()
    #compute loss
    loss = loss_fn(yhat,y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    #optimizer.cleargrads()
    yhatsig = torch.sigmoid(yhat)
    acc = accuracy(yhatsig, y)

    return loss, acc
  return train_step

def accuracy(preds, labels):
    acc = 0
    for i, pred in enumerate(preds):
        p = torch.argmax(pred)
        
        if torch.round(pred) == labels[i]:
            acc +=1

    return acc/len(preds)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.resnet18(pretrained=True)

#freeze all params
# for params in model.parameters():
#   params.requires_grad_ = False

#add a new final layer
nr_filters = model.fc.in_features  #number of input features of last layer
model.fc = nn.Linear(nr_filters, 1)

model = model.to(device)
from torch.nn.modules.loss import BCEWithLogitsLoss
from torch.optim import lr_scheduler

#loss
loss_fn = BCEWithLogitsLoss() #binary cross entropy with sigmoid, so no need to use sigmoid in the model

#optimizer
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.05, betas=(0.9, 0.999))

#train step
train_step = make_train_step(model, optimizer, loss_fn)

from tqdm import tqdm

losses = []
val_losses = []
accs = []
val_accs = []
epoch_train_losses = []
epoch_test_losses = []
epoch_train_accs = []
epoch_test_accs = []

n_epochs = 100
early_stopping_tolerance = 5
early_stopping_threshold = 0.03

for epoch in range(n_epochs):
  epoch_loss = 0
  epoch_acc = 0

  for i ,data in tqdm(enumerate(trainloader), total = len(trainloader)): #iterate ove batches
    x_batch , y_batch = data
    x_batch = x_batch.to(device) #move to gpu
    y_batch = y_batch.unsqueeze(1).float() #convert target to same nn output shape
    y_batch = y_batch.to(device) #move to gpu


    loss, acc = train_step(x_batch, y_batch)
    epoch_acc += acc/len(trainloader)
    epoch_loss += loss/len(trainloader)
    losses.append(loss)
    accs.append(acc)

  epoch_train_losses.append(epoch_loss)
  epoch_train_accs.append(epoch_acc)

  print('\nEpoch : {}, train loss : {}, train acc : {}'.format(epoch+1,epoch_loss, epoch_acc))

  #validation doesnt requires gradient
  with torch.no_grad():
    cum_loss = 0
    cum_acc = 0
    for x_batch, y_batch in testloader:
      x_batch = x_batch.to(device)
      y_batch = y_batch.unsqueeze(1).float() #convert target to same nn output shape
      y_batch = y_batch.to(device)

      #model to eval mode
      model.eval()
      
      yhat = model(x_batch)
      yhatsig = torch.sigmoid(yhat)
      # for i in range(len(yhatsig)):
      #   print(yhatsig[i], y_batch[i])
    
      val_loss = loss_fn(yhat,y_batch)
      cum_loss += loss/len(testloader)
      val_losses.append(val_loss.item())

      val_acc = accuracy(yhatsig, y_batch)
      cum_acc += val_acc/len(testloader)
      val_accs.append(val_acc)


    epoch_test_losses.append(cum_loss)
    epoch_test_accs.append(cum_acc)

    print('Epoch : {}, val loss : {} val acc: {}'.format(epoch+1,cum_loss, cum_acc))

    best_loss = min(epoch_test_losses)
    best_acc = min(epoch_test_accs)
    #save best model
    if cum_acc <= best_acc:
      best_model_wts = model.state_dict()

    #early stopping
    early_stopping_counter = 0
    if cum_acc > best_acc:
      early_stopping_counter +=1

    if (early_stopping_counter == early_stopping_tolerance) or (best_acc <= early_stopping_threshold):
      print("/nTerminating: early stopping")
      break #terminate training

#load best model
model.load_state_dict(best_model_wts)


100%|██████████| 14/14 [00:01<00:00,  7.05it/s]



Epoch : 1, train loss : 3.421875476837158, train acc : 0.5530563186813187
Epoch : 1, val loss : 1.2366751432418823 val acc: 0.765625


100%|██████████| 14/14 [00:01<00:00,  7.19it/s]



Epoch : 2, train loss : 0.6084225177764893, train acc : 0.7034684065934068
Epoch : 2, val loss : 0.49452459812164307 val acc: 0.7361111111111112


100%|██████████| 14/14 [00:01<00:00,  7.25it/s]


...

100%|██████████| 14/14 [00:02<00:00,  6.37it/s]



Epoch : 99, train loss : 2.5760979652404785, train acc : 0.7589285714285715
Epoch : 99, val loss : 0.0004654668446164578 val acc: 0.8038194444444444


100%|██████████| 14/14 [00:02<00:00,  6.37it/s]



Epoch : 100, train loss : 2.5849530696868896, train acc : 0.7544642857142857
Epoch : 100, val loss : 0.00045253196731209755 val acc: 0.8038194444444444


<All keys matched successfully>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')